In [1]:
import os
import math
import operator
import numpy as np
import random

In [2]:
import csv

with open('sentiment_mturk_answers.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile))
data = np.array(data)
data = data[1:,:]

data = data[:1000]

In [11]:
def listOfInput(data):
    unique_list = [] 
      
    # traverse for all elements 
    for x in data: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list

def trueLabelList(inp, data, l):
    result = []
    
    for i in inp:
        for j in range(len(data)):
            if(i == data[j,l]):
                result.append([i,data[j,-2]])
                break;
    return result

def numlized(label):
    for i in range(len(label)):
        if label[i,-1] == "pos":
            label[i,-1] = 1
        else:
            label[i,-1] = 0
    return label
            
def posteriorP(L, i, label):
    pos = 0
    neg = 0
    
    for x in L[i]:
        if x == 1:
            pos+=1
        elif x == 0:
            neg+=1
    
    if pos+neg == 0:
        return 0
    
    if label == 1:
        return pos/(pos+neg)
    else:
        return neg/(pos+neg)
    
def predPz(L,i,a,label):
    allL = L[i]
    z = 0.5
    
    for j in range(len(allL)):
        if allL[j]!= -1:
            z *= a[j]
    print(z)
    z *= posteriorP(L,i,label)
    if z > 1:
        z = 1/z
    return z

def calcQ(L,i,a):
    allL = L[i]
    z = 0
    
    for j in range(len(allL)):
        if allL[j]!= -1:
            if a[j] > 0:
                z += np.log(a[j])
    return abs(z)
    
def getTrueLabel(data, t):
    for i in range(len(data)):
        if t == data[i,0]:
            return data[i,1]

def findLabel(data,A,image):
    for i in range(len(data)):
        if(data[i,0] == A):                # !! assume Annotator id in colomn 0
            if(data[i,1] == image):        # !! assume image in colomn 1
                return data[i,-1]          # !! assume predict label in colomn -1 
    return -1
    
def askExpert(L,I,A,E,n,data):
    success = 0
    
    for i in range(len(A)):
        if A[i] in E:
            if L[n,i] == -1:
                if findLabel(data,A[i],I) != -1:
                    L[n,i] =  findLabel(data,A[i],I)
                    success = 1
    return (L,success)

def askBot(L,I,A,E,n,data):
    success = 0
    
    for i in range(len(A)):
        if A[i] not in E:
            if L[n,i] == -1:
                if findLabel(data,A[i],I) != -1:
                    L[n,i] =  findLabel(data,A[i],I)
                    success = 1
    return (L,success)

    
def BAM(image, A, trueLabel,data):  # image is all images, A is all annotators
    I = []       # images which labeled at least once collected
    L = np.zeros((len(image),len(A))) -1      # all labels
    E = []      #  expert-list
    B = []      #  bot-list
    n = 50      # each round picture adding number
    a = np.random.uniform(0, 1,len(A))     # intialized the reliability of every annotator
    z = [0] * len(image) # unknow target value for each i image in I
    r = 0.8        #threshold
    ex = 0.7      #expert threshold
    
    
    while (len(I) < len(image)):
        
        print(len(I))
        
        for idx in range(n):
                for x in image:
                    if x not in I:
                        I.append(x)
                        break
                
                if len(I) >= len(image):
                    break
        
        for i in range(len(I)):
            pz = predPz(L,i,a,getTrueLabel(trueLabel,I[i]))
            
            while pz < r:
                expLabel = askExpert(L,I[i],A,E,i,data)
                
                if expLabel[1] == 1:
                    L = expLabel[0]
                    pz = predPz(L,i,a,getTrueLabel(trueLabel,I[i]))
                else:
                    botLabel = askBot(L,I[i],A,E,i,data)
                    if botLabel[1] == 0:
                        break
                    else:
                        L = botLabel[0]
                        pz = predPz(L,i,a,getTrueLabel(trueLabel,I[i]))
            z[i] = pz
         
        print("finish label")
        
        E = [] 
        B = []
        for i in range(len(A)):
            if a[i] > 0:
                a[i] = np.log(a[i]) + calcQ(L,i,a)
            else:
                a[i] = calcQ(L,i,a)
            a[i] = abs(a[i])
            #if a[i] > 1:
                #a[i] = 1/a[i]
            if a[i] > ex:
                E.append(A[i])
            else:
                B.append(A[i])
                
        print("finish Ano")
        
    return z,a

In [4]:
image = listOfInput(data[:,1])
len(image)

['4518',
 '10415',
 '7098',
 '4396',
 '2812',
 '7861',
 '5133',
 '8474',
 '8774',
 '9387',
 '4491',
 '7868',
 '4390',
 '7442',
 '10425',
 '3143',
 '3199',
 '2696',
 '4334',
 '6278',
 '3824',
 '561',
 '8307',
 '3645',
 '3149',
 '8838',
 '8155',
 '1521',
 '1922',
 '5769',
 '213',
 '1654',
 '9473',
 '907',
 '10054',
 '8446',
 '4632',
 '4028',
 '9482',
 '5402',
 '5998',
 '3550',
 '3696',
 '9209',
 '9626',
 '7584',
 '3922',
 '411',
 '4400',
 '8583',
 '6900',
 '8014',
 '7143',
 '8640',
 '4283',
 '1049',
 '2765',
 '10355',
 '9695',
 '404',
 '7943',
 '2413',
 '4233',
 '7074',
 '5564',
 '5698',
 '10163',
 '2184',
 '1269',
 '9385',
 '6479',
 '2567',
 '2191',
 '6852',
 '8497',
 '74',
 '4347',
 '278',
 '3418',
 '10034',
 '6002',
 '2952',
 '4979',
 '1712',
 '1542',
 '1634',
 '7916',
 '10314',
 '3899',
 '1519',
 '2866',
 '2284',
 '3859',
 '1648',
 '1892',
 '7681',
 '9153',
 '7533',
 '5009',
 '9914',
 '3972',
 '3412',
 '8097',
 '994',
 '9293',
 '8535',
 '4443',
 '2324',
 '1106',
 '8931',
 '5137',
 '2

In [5]:
Anotators = listOfInput(data[:,0]) 
len(Anotators)

['A2HD5XMM48KKJW',
 'A233ONYNWKDIYF',
 'A33SMNMTMIOJ6T',
 'A24ZS6FANCA3E5',
 'A2865KTIP7MIXZ',
 'A207OR9LV0PAPY',
 'ARX0S1CIDJLOX',
 'A1JH7KENWHOGB7',
 'A2T19IADVP11HY',
 'APPTLVXZD0SKE',
 'A119KIC2YMT1HX',
 'A31A20Z5P0UJBH',
 'A1UB330C1JPJ9I',
 'A3CFNUD7VR2E1E',
 'ANCF8G8GCO4EM',
 'AL12RG9EJZ60',
 'A3TPYQRWF3QML7',
 'A8028AFBBS29G',
 'AFDMO7SSL5F3V',
 'A111JI6APXR6QV',
 'A3A5BAO5NT2W8',
 'A3SXLBX06D9FP9',
 'AD5IKFAK8KN4P',
 'A79CHO3NW2IPO',
 'A1F4D2PZ7NNWTL',
 'AGBA9QNP2BKR1',
 'A37OZD4L9W21QK',
 'A2TEWO3WJ5YKGG',
 'A3HTLXAPOBCVVO',
 'A1ZZINLCZZYOH4',
 'AD1ILDUXZHASF',
 'A28TXBSZPWMEU9',
 'A18TDHUVOMVJRI',
 'A35OGKV242IJ78',
 'A1WBR902VW7HTY',
 'A122JE350RXCLB',
 'A3CJ3U84B1VDOU',
 'A1E90HXPEL34ZU',
 'A36KE97IOQINHY',
 'A3SWVCOJPJM7KQ',
 'AQN3RS63MEPQA',
 'A36P6YLIJXYPHA',
 'A3RJ8NZQ8BRTZM',
 'A1JWEUVNAZ6WVA']

In [6]:
trueLabel = trueLabelList(image, data, 1)
trueLabel = np.array(trueLabel)

In [7]:
numlized(trueLabel)

array([['4518', '1'],
       ['10415', '0'],
       ['7098', '0'],
       ['4396', '1'],
       ['2812', '1'],
       ['7861', '0'],
       ['5133', '1'],
       ['8474', '0'],
       ['8774', '0'],
       ['9387', '0'],
       ['4491', '1'],
       ['7868', '0'],
       ['4390', '1'],
       ['7442', '0'],
       ['10425', '0'],
       ['3143', '1'],
       ['3199', '1'],
       ['2696', '1'],
       ['4334', '1'],
       ['6278', '0'],
       ['3824', '1'],
       ['561', '1'],
       ['8307', '0'],
       ['3645', '1'],
       ['3149', '1'],
       ['8838', '0'],
       ['8155', '0'],
       ['1521', '1'],
       ['1922', '1'],
       ['5769', '0'],
       ['213', '1'],
       ['1654', '1'],
       ['9473', '0'],
       ['907', '1'],
       ['10054', '0'],
       ['8446', '0'],
       ['4632', '1'],
       ['4028', '1'],
       ['9482', '0'],
       ['5402', '0'],
       ['5998', '0'],
       ['3550', '1'],
       ['3696', '1'],
       ['9209', '0'],
       ['9626', '0'],
       ['7

In [8]:
numlized(data)

array([['A2HD5XMM48KKJW', '4518',
        'the cast is phenomenal , especially the women . ',
        'cast phenomen especi women', 'pos', '1'],
       ['A2HD5XMM48KKJW', '10415',
        'the metaphors are provocative , but too often , the viewer is left puzzled by the mechanics of the delivery . ',
        'metaphor provoc often viewer left puzzl mechan deliveri', 'neg',
        '0'],
       ['A2HD5XMM48KKJW', '7098',
        "while there's something intrinsically funny about sir anthony hopkins saying 'get in the car , bitch , ' this jerry bruckheimer production has little else to offer",
        'there someth intrins funni sir anthoni hopkin sai car bitch jerri bruckheim product littl els offer',
        'neg', '1'],
       ...,
       ['A28TXBSZPWMEU9', '5124',
        'though the aboriginal aspect lends the ending an extraordinary poignancy , and the story itself could be played out in any working class community in the nation . ',
        'though aborigin aspect lend end extraor

In [13]:
bam = BAM(image, Anotators, trueLabel,data)

0
0.5
0.011002486585886425
0.5
0.011002486585886425
0.5
0.011002486585886425
0.5
0.017460021731198715
0.5
0.011002486585886425
0.5
2.9060608424999485e-05
0.5
2.9060608424999485e-05
0.5
2.9060608424999485e-05
0.5
2.9060608424999485e-05
0.5
2.9060608424999485e-05
0.5
5.785348895665549e-08
0.5
5.785348895665549e-08
0.5
5.785348895665549e-08
0.5
5.785348895665549e-08
0.5
5.785348895665549e-08
0.5
7.951749863885499e-06
0.5
7.951749863885499e-06
0.5
7.951749863885499e-06
0.5
7.951749863885499e-06
0.5
7.951749863885499e-06
0.5
6.79531053825656e-06
0.5
6.79531053825656e-06
0.5
6.79531053825656e-06
0.5
6.79531053825656e-06
0.5
6.79531053825656e-06
0.5
8.968068556426884e-05
0.5
8.968068556426884e-05
0.5
8.968068556426884e-05
0.5
8.968068556426884e-05
0.5
8.968068556426884e-05
0.5
1.89769479206847e-05
0.5
1.89769479206847e-05
0.5
1.89769479206847e-05
0.5
1.89769479206847e-05
0.5
1.89769479206847e-05
0.5
0.00015777119859706898
0.5
0.00015777119859706898
0.5
0.00015777119859706898
0.5
0.00015777119

In [14]:
bam

([0.0,
  0.00012269961867780756,
  0.0002044993644630126,
  0.003793239960941968,
  0.0003067490466945189,
  4.2224950081739384e-05,
  0.0,
  4.2224950081739384e-05,
  4.2224950081739384e-05,
  4.2224950081739384e-05,
  0.0,
  9.26583225412958e-06,
  0.0,
  6.949374190597185e-06,
  5.559499352477749e-06,
  2.0523389883597554e-05,
  0.0,
  0.0,
  0.0,
  5.130847470899389e-06,
  0.0,
  0.0,
  4.663694482510264e-06,
  0.0,
  2.3318472412551314e-05,
  5.558034681905549e-06,
  5.558034681905549e-06,
  0.0,
  0.0,
  5.558034681905549e-06,
  0.0,
  0.0,
  3.991786444191603e-06,
  0.0,
  3.991786444191603e-06,
  6.47369858952553e-06,
  1.942109576857659e-05,
  0.0,
  9.710547884288295e-06,
  4.855273942144148e-06,
  6.85444353922693e-06,
  0.0,
  0.0,
  0.0,
  6.85444353922693e-06,
  1.7226259415372685e-05,
  5.167877824611804e-05,
  2.583938912305902e-05,
  1.291969456152951e-05,
  1.291969456152951e-05,
  1.4014777781037526e-05,
  1.7518472226296905e-05,
  1.4014777781037526e-05,
  1.4014777